In [2]:
3 + 8

11

In [3]:
3**4

81

In [4]:
4*(7 - 2)

20

In [5]:
a = 2
b = 11
c = a + b
print(c)

13


In [6]:
# Long-hand addition
a = 1
a = a + 4
print(a)

# Short-hand addition
a = 1
a += 4
print(a)

5
5


In [7]:
from simtk.openmm.app import PDBFile
from openforcefield.utils import get_data_filename

Unable to load toolkit <openforcefield.utils.toolkits.OpenEyeToolkitWrapper object at 0x7efc7fb1b278>.


No module named 'openeye'


In [10]:
# 1 molecule of ethanol and 1 of cyclohexane.
pdb_file_path = get_data_filename('systems/test_systems/1_cyclohexane_1_ethanol.pdb')

# 40%-60% cyclohexane-ethanol mixture.
# pdb_file_path = get_data_filename('systems/packmol_boxes/cyclohexane_ethanol_0.4_0.6.pdb')

pdbfile = PDBFile(pdb_file_path)

In [11]:
from openforcefield.topology import Molecule
ethanol = Molecule.from_smiles('CCO')
cyclohexane = Molecule.from_smiles('C1CCCCC1')

In [12]:
from openforcefield.topology import Topology
from openforcefield.typing.engines.smirnoff import ForceField

# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[ethanol, cyclohexane])

# Load the smirnof99Frosst force field.
forcefield = ForceField('smirnoff99Frosst.offxml')

# Parametrize the topology and create an OpenMM System.
system = forcefield.create_openmm_system(off_topology)

In AmberToolsToolkitwrapper.computer_partial_charges_am1bcc: Molecule '' has more than one conformer, but this function will only generate charges for the first one.
In AmberToolsToolkitwrapper.computer_partial_charges_am1bcc: Molecule '' has more than one conformer, but this function will only generate charges for the first one.


In [13]:
from simtk import openmm, unit

# Propagate the System with Langevin dynamics.
time_step = 2*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

# Set up an OpenMM simulation.
simulation = openmm.app.Simulation(omm_topology, system, integrator)

# Set the initial positions.
positions = pdbfile.getPositions() 
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# Configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [14]:
import time

print("Starting simulation")
start = time.clock()

# Run the simulation
simulation.step(num_steps)

end = time.clock()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")

Starting simulation


/srv/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  after removing the cwd from sys.path.


Elapsed time 19.02 seconds
Done!


/srv/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':
